# Exploratory Data Analysis

Hypothesis: We can use the https://arxiv.org/pdf/2406.04313 method for increasing honesty

In [1]:
# autoreload your package
%load_ext autoreload
%autoreload 2
import adapter_overseer


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
import warnings
# warnings.simplefilter("ignore")
# warnings.filterwarnings("ignore", ".*does not have many workers.*")
# warnings.filterwarnings("ignore", ".*divide by zero.*")

## numeric, plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (7.0, 4)

## utils
from pathlib import Path
from tqdm.auto import tqdm
import logging, os, re
import collections, functools, itertools
from loguru import logger

from typing import List, Callable, Tuple, Dict, Optional
from jaxtyping import Float, Int
from torch import Tensor

# torch
# import pytorch_lightning as pl
from einops import rearrange, repeat, reduce
import torch
import torch.nn as nn


from baukit.nethook import get_module
from baukit import TraceDict

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from adapter_overseer.config import ExtractConfig

cfg = ExtractConfig()
cfg

ExtractConfig(datasets=('amazon_polarity',), datasets_ood='imdb', model='failspy/Llama-3-8B-Instruct-abliterated', collection_layers=('base_model.model.model.layers.10', 'base_model.model.model.layers.20'), batch_size=2, prompt_format=None, num_shots=2, max_length=776, max_examples=1000, seed=42, max_epochs=1)

## Load

In [5]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# https://huggingface.co/blog/mlabonne/orpo-llama-3
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
torch_dtype, device

(torch.bfloat16, device(type='cuda', index=0))

In [7]:
# load model
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
quantization_config = BitsAndBytesConfig(load_in_4bit=True,     bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch_dtype, bnb_4bit_use_double_quant=True,)
model = AutoModelForCausalLM.from_pretrained(cfg.model, device_map="auto", quantization_config=quantization_config,)
model

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.72s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [8]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'
tokenizer.truncation_side = 'left'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# https://old.reddit.com/r/LocalLLaMA/comments/1coizjy/tokenizer_config_of_llama3_changed_by_meta_in_hf/
tokenizer.eos_token # it's good

'<|eot_id|>'

In [10]:
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# \peft_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
# )
# model = get_peft_model(model, peft_config)


In [11]:
# from peft import prepare_model_for_int8_training
# # we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in float32 for stability. We also cast the output of the last layer in float32 for the same reasons.
# model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

In [12]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model
# https://github.com/huggingface/peft/blob/main/src/peft/utils/constants.py
config = LoraConfig(
                        #r=32, lora_alpha=64, 
                    # target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none"
                    )


# LoRA config
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
# )
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, config)
model.print_trainable_parameters()

with model.disable_adapter():
    model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


## Get data

In [13]:
# perhaps use load_preproc_datasets from sdb_probes_are_lie_detectors repo... /media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/src/prompts/prompt_loading.py

In [14]:
# load a dataset of paired prompts, to try and get the model to lie
from adapter_overseer.prompts.prompt_loading import load_preproc_datasets

N = cfg.max_examples
ds_tokens = load_preproc_datasets(
    cfg.datasets,
    tokenizer,
    N=N,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
    max_length=cfg.max_length,
    prompt_format=cfg.prompt_format,
)
ds_tokens


2024-06-10 07:40:28.142 | INFO     | adapter_overseer.prompts.prompt_loading:load_preproc_dataset:392 - median token length: 375.0 for amazon_polarity. max_length=776
2024-06-10 07:40:28.144 | INFO     | adapter_overseer.prompts.prompt_loading:load_preproc_dataset:396 - truncation rate: 0.00% on amazon_polarity
/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:2328: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(
2024-06-10 07:40:28.425 | INFO     | adapter_overseer.prompts.prompt_loading:load_preproc_dataset:405 - num_rows (after filtering out truncated rows) 3004=>3004


Dataset({
    features: ['ds_string', 'example_i', 'answer', 'messages', 'answer_choices', 'template_name', 'label_true', 'label_instructed', 'instructed_to_lie', 'sys_instr_name', 'question', 'input_ids', 'attention_mask', 'truncated', 'length', 'prompt_truncated', 'choice_ids'],
    num_rows: 1001
})

## Train: transformers

https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_opt_bnb_peft.ipynb

In [15]:
# TODO change the loss function!
# we need to modify the forward pass, so that it returns a different loss function
# but to calculate this we will need to residuals now, and as they werre
# loss_bad = mse(repr_current, repr_target)

# from transformers import SFTTrainer
from trl.trainer import SFTTrainer, SFTConfig
import torch.nn.functional as F

from adapter_overseer.helpers.torch_helpers import clear_mem, switch
from adapter_overseer.helpers.scores import select_choices

class CustomSFTTrainer(SFTTrainer):
    """
    Custom SFTTrainer that orthoganalizes the repr of bad examples, and retains good repr of examples

    See: https://arxiv.org/pdf/2406.04313

    args:
        collection_layers: list of baukit layer names to collect
    """
    def __init__(self, *args, collection_layers: list, alpha=0.1, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.collection_layers = collection_layers
        self.alpha = alpha
        self.total_steps = self.args.max_steps

    def compute_loss(self, model, inputs, return_outputs=False):       

        batch = {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}

        # collect the residuals of the model
        with model.disable_adapter():
            orig_outputs = model(**batch, output_hidden_states=True)
        outputs = model(**batch, output_hidden_states=True)

        def collect_hs(hidden_states):
            """The residual stream is the diff of the hs."""
            hs = [hidden_states[i] for i in self.collection_layers]
            return rearrange(hs, 'l b t h -> b l t h').diff(1)

        rep_adapt = collect_hs(outputs.hidden_states)
        rep_orig = collect_hs(orig_outputs.hidden_states)

        # so now we have a mixed batch of good and bad outputs
        # get probs of each choice
        # compare to labels to seperate into good and bad
        choice_ids = inputs['choice_ids'].detach().cpu().long()
        # label_instructed = inputs['label_true'] ^ inputs['instructed_to_lie']
        label_true = inputs['label_true']

        # does the underlying model get it right or wrong?
        end_logits = orig_outputs["logits"][:, -1]
        probs = torch.softmax(end_logits, -1)
        choice_probs = select_choices(probs, choice_ids).sum(2)
        binary_ans = choice_probs[:, 1] / (choice_probs.sum(1) + 1e-12)
        correct_truth_telling = switch(binary_ans, label_true)
        # correct_instruction_following = switch(binary_ans, label_instructed)

        mask_desired = correct_truth_telling>0.5


        # get coeffecient
        steps = self.state.global_step + 1
        c = torch.tensor(self.alpha * steps / (2 * self.total_steps)).to(rep_orig.dtype)
        loss_retain = F.mse_loss(rep_orig, rep_adapt, reduction='none' )[mask_desired]
        if loss_retain.numel() == 0:
            loss_retain = 0
        else:
            loss_retain = loss_retain.mean()
        loss_rr = F.relu(F.cosine_similarity(rep_orig, rep_adapt, dim=1))[~mask_desired]
        if loss_rr.numel() == 0:
            loss_rr = 0
        else:
            loss_rr = loss_rr.mean()
        loss = loss_rr * (1 - c) + c * loss_retain
        loss = loss
        logger.debug(f"steps: {steps}, c: {c}, loss_rr: {loss_rr:2.3f}, loss_retain: {loss_retain:2.3f}, loss={loss:2.3f}, mask_desired: {(mask_desired*1.0).mean():2.3f}")
        
        return (loss, outputs) if return_outputs else loss
    

# TODO make sure that multiple cols get passed into trainer
ds = ds_tokens.select_columns(['label_true', 'label_instructed' ,'instructed_to_lie', 'input_ids', 'attention_mask', 'choice_ids'])

import transformers

# see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py#L58
trainer = CustomSFTTrainer(
    model=model,
    train_dataset=ds,
    collection_layers=[10, 20],
    # max_seq_length=cfg.max_length,
    args=SFTConfig(
        # see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_config.py#L21
        max_seq_length=cfg.max_length,
        per_device_train_batch_size=4, # 18GB/24GB
        gradient_accumulation_steps=6, # we want to accumulate the gradients to make the batch size larger, so we have sufficient examples of good and bad behaviour to learn from
        warmup_steps=10,
        max_steps=2000,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        remove_unused_columns=False,
    ),
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
max_steps is given, it will override any value given in num_train_epochs
  0%|          | 0/2000 [00:00<?, ?it/s]/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
2024-06-10 07:40:31.910 | DEBUG    | __main__:compute_loss:78 - steps: 1, c: 2.499999936844688e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.500
2024-06-10 07:40:37.111 | DEBUG    | __main__:compute_loss:78 - steps: 1, c: 2.4

{'loss': 1.0, 'grad_norm': 3.3594227399902366e-10, 'learning_rate': 2.9999999999999997e-05, 'epoch': 0.02}


2024-06-10 07:41:03.040 | DEBUG    | __main__:compute_loss:78 - steps: 2, c: 4.999999873689376e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.500
2024-06-10 07:41:08.245 | DEBUG    | __main__:compute_loss:78 - steps: 2, c: 4.999999873689376e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.500
2024-06-10 07:41:13.463 | DEBUG    | __main__:compute_loss:78 - steps: 2, c: 4.999999873689376e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.500
2024-06-10 07:41:18.703 | DEBUG    | __main__:compute_loss:78 - steps: 2, c: 4.999999873689376e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.500
2024-06-10 07:41:23.954 | DEBUG    | __main__:compute_loss:78 - steps: 2, c: 4.999999873689376e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.000
2024-06-10 07:41:29.223 | DEBUG    | __main__:compute_loss:78 - steps: 2, c: 4.999999873689376e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired

{'loss': 1.0, 'grad_norm': 6.474682595625225e-11, 'learning_rate': 5.9999999999999995e-05, 'epoch': 0.05}


2024-06-10 07:41:34.526 | DEBUG    | __main__:compute_loss:78 - steps: 3, c: 7.500000356230885e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.500
2024-06-10 07:41:39.828 | DEBUG    | __main__:compute_loss:78 - steps: 3, c: 7.500000356230885e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.750
2024-06-10 07:41:45.140 | DEBUG    | __main__:compute_loss:78 - steps: 3, c: 7.500000356230885e-05, loss_rr: 0.000, loss_retain: 0.000, loss=0.000, mask_desired: 1.000
2024-06-10 07:41:50.460 | DEBUG    | __main__:compute_loss:78 - steps: 3, c: 7.500000356230885e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.750
2024-06-10 07:41:55.798 | DEBUG    | __main__:compute_loss:78 - steps: 3, c: 7.500000356230885e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired: 0.500
2024-06-10 07:42:01.164 | DEBUG    | __main__:compute_loss:78 - steps: 3, c: 7.500000356230885e-05, loss_rr: 1.000, loss_retain: 0.000, loss=1.000, mask_desired

...

{'loss': 0.3334, 'grad_norm': 1.5648552179336548, 'learning_rate': 0.0002968341708542713, 'epoch': 0.74}


2024-06-10 07:57:48.571 | DEBUG    | __main__:compute_loss:78 - steps: 32, c: 0.0007999999797903001, loss_rr: 0.323, loss_retain: 0.059, loss=0.323, mask_desired: 0.250
2024-06-10 07:57:54.241 | DEBUG    | __main__:compute_loss:78 - steps: 32, c: 0.0007999999797903001, loss_rr: 0.323, loss_retain: 0.061, loss=0.323, mask_desired: 0.250
2024-06-10 07:57:59.910 | DEBUG    | __main__:compute_loss:78 - steps: 32, c: 0.0007999999797903001, loss_rr: 0.323, loss_retain: 0.059, loss=0.323, mask_desired: 0.750
2024-06-10 07:58:05.557 | DEBUG    | __main__:compute_loss:78 - steps: 32, c: 0.0007999999797903001, loss_rr: 0.321, loss_retain: 0.058, loss=0.321, mask_desired: 0.750
2024-06-10 07:58:11.208 | DEBUG    | __main__:compute_loss:78 - steps: 32, c: 0.0007999999797903001, loss_rr: 0.310, loss_retain: 0.056, loss=0.309, mask_desired: 0.500
2024-06-10 07:58:16.866 | DEBUG    | __main__:compute_loss:78 - steps: 32, c: 0.0007999999797903001, loss_rr: 0.324, loss_retain: 0.054, loss=0.324, mask_d

{'loss': 0.3204, 'grad_norm': 1.7377934455871582, 'learning_rate': 0.00029668341708542713, 'epoch': 0.76}


2024-06-10 07:58:22.537 | DEBUG    | __main__:compute_loss:78 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.309, loss_retain: 0.000, loss=0.309, mask_desired: 0.000
2024-06-10 07:58:28.213 | DEBUG    | __main__:compute_loss:78 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.308, loss_retain: 0.060, loss=0.307, mask_desired: 0.250
2024-06-10 07:58:33.875 | DEBUG    | __main__:compute_loss:78 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.316, loss_retain: 0.059, loss=0.315, mask_desired: 0.250
2024-06-10 07:58:39.540 | DEBUG    | __main__:compute_loss:78 - steps: 33, c: 0.0008249999955296516, loss_rr: 0.311, loss_retain: 0.061, loss=0.310, mask_desired: 0.750


KeyboardInterrupt: 

In [ ]:
# save
model.save_pretrained("../outputs/hs_adapter")

# Eval

In [37]:
from datasets import load_dataset
# multiple_choice
from torch.utils.data import DataLoader
# dataset = load_dataset("truthfulqa/truthful_qa", "multiple_choice")

# HACK it was stalling for hours, so I loaded it locally
dataset = load_dataset("../data/truthful_qa")['validation']
dataset

Dataset({
    features: ['question', 'mc1_targets', 'mc2_targets'],
    num_rows: 817
})

In [96]:

# print(row)

def format_prompt(row):
    prompt = f"Q: {row['question']}\n"
    for i, choice in enumerate(row['mc1_targets']['choices']):
        prompt += f"{i+1}. {choice}\n"

    choices = [str(i) for i in range(len(row['mc1_targets']['labels']))]
    return {'text': prompt, 
            'label': [np.argmax(row['mc1_targets']['labels'])],
            'choices': choices,
            'num_choices': len(choices),
            }

dataset1 = dataset.map(format_prompt)

Map: 100%|██████████| 817/817 [00:00<00:00, 6091.51 examples/s]


In [103]:
max([len(r['labels']) for r in dataset['mc1_targets']])

13

In [108]:
# get our choice ids
choices = [str(i) for i in range(13)]
choice_ids = [tokenizer(c, add_special_tokens=False).input_ids[0] for c in choices]
choice_ids

[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 605, 806, 717]

In [109]:

def tokenization(example):
    o = tokenizer(example["text"], padding="max_length", truncation=True, max_length=cfg.max_length, return_tensors="pt")
    return o

dataset2 = dataset1.map(tokenization, batched=True).select_columns([ 'label', 'input_ids', 'attention_mask', 
                                                                    'num_choices'
                                                                    ]).with_format("torch")
dataset2

Map: 100%|██████████| 817/817 [00:00<00:00, 4213.33 examples/s]


Dataset({
    features: ['label', 'input_ids', 'attention_mask', 'num_choices'],
    num_rows: 817
})

In [168]:
# https://github.dev/sylinrl/TruthfulQA/blob/fdd8ad1c0d00a478cf8b0bb41a3ad8378c16293b/truthfulqa/models.py#L311


probs = []
base_probs = []

dl = DataLoader(
    dataset2, batch_size=4, num_workers=0)
for b in tqdm(dl):
    inputs = {'input_ids': b['input_ids'], 'attention_mask': b['attention_mask']}
    with torch.no_grad():
        with model.disable_adapter():
            out_base = model(**inputs)
        out = model(**inputs)

        for j in range(len(out["logits"])):
            n = b['num_choices'][j]
            b_choice_ids = choice_ids[:n]
            label = b['label'][j, 0]

            choice_probs_base = out_base["logits"][j, -1, b_choice_ids].softmax(dim=-1)
            choice_probs = out["logits"][j, -1, b_choice_ids].softmax(dim=-1)
            prob = choice_probs[label].item()
            prob_base = choice_probs_base[label].item()
            probs.append(prob)
            base_probs.append(prob_base)
        

/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/media/wassname/SGIronWolf/projects5/elk/adapters_can_monitor_lies/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  1%|          | 2/205 [00:12<20:23,  6.03s/it]


KeyboardInterrupt: 

In [172]:
choice_probs

tensor([0.0395, 0.5892, 0.2594, 0.0377, 0.0161, 0.0065, 0.0130, 0.0232, 0.0068,
        0.0022, 0.0064])

In [173]:
choice_probs_base

tensor([0.0395, 0.5892, 0.2594, 0.0377, 0.0161, 0.0065, 0.0130, 0.0232, 0.0068,
        0.0022, 0.0064])

In [169]:
acc = ((torch.tensor(probs)>0.5)*1.0).mean()
base_acc = ((torch.tensor(base_probs)>0.5)*1.0).mean()
acc, base_acc

(tensor(0.), tensor(0.))

In [171]:
prob_correct = torch.tensor(probs).mean()
prob_base_correct = torch.tensor(base_probs).mean()
prob_correct, prob_base_correct

(tensor(0.0139), tensor(0.0139))

torch.Size([4, 13])

In [112]:
b

{'label': tensor([[0],
         [0],
         [0],
         [0]]),
 'input_ids': tensor([[128009, 128009, 128009,  ...,   3723,   4273,    627],
         [128009, 128009, 128009,  ...,     13,   8494,    627],
         [128009, 128009, 128009,  ...,    559,   9949,    627],
         [128009, 128009, 128009,  ...,    304,  16759,    627]]),
 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1]]),
 'num_choices': tensor([4, 5, 4, 4])}

In [ ]:
from datasets import load_dataset
# import evaluate
from transformers import pipeline

# accuracy = evaluate.load("accuracy")

dataset_eval = load_dataset("EleutherAI/truthful_qa_binary", split="test")["text"]

In [ ]:
from evaluate import evaluator
from datasets import load_dataset
data = load_dataset("EleutherAI/truthful_qa_binary", split="validation[:20]")
task_evaluator = evaluator("text-classification")
# see https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/evaluator_classes#evaluate.TextClassificationEvaluator

pipe = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    # feature_extractor=feature_extractor,
    # device=device,
)


results = task_evaluator.compute(
    pipe,
    # tokenizer=tokenizer,
    data=data,
    metric="accuracy",
    input_column="question",
    # label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    random_state=42
)

In [ ]:

# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True)

# https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/pipelines#pipeline-batching
pipe = pipeline("text-classification", device="auto", model=model, tokenizer=tokenizer)
batch_size = 4
for out in tqdm(pipe(dataset_eval, batch_size=batch_size), total=len(dataset_eval)):
    1/0
    pass

In [176]:
model.train()
model.print_trainable_parameters()

trainable params: 0 || all params: 8,033,669,120 || trainable%: 0.0000


In [177]:
with model.disable_adapter():
    model.print_trainable_parameters()

trainable params: 0 || all params: 8,033,669,120 || trainable%: 0.0000
